In [ ]:
import os
from utils import *
from agents import *
import time
import torch
import torch.nn as nn
from copy import deepcopy
import argparse

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--seeds',          type=int,       default=[2020, 2021, 2022, 2023, 2024, 2025])
parser.add_argument('--num_epochs',     type=int,       default=100)
parser.add_argument('--lr',             type=float,     default=0.005)
parser.add_argument('--dataset',        type=str,       default='fmnist')
parser.add_argument('--batch_size',     type=int,       default=256)
parser.add_argument('--model_name',     type=str,       default='alexnet')
parser.add_argument('--repeat',         type=int,       default=3)
parser.add_argument('--patience',       type=int,       default=25)
parser.add_argument('--unlearn_class',  type=list,      default=3)
args = parser.parse_args("")
args.time_str = time.strftime("%m-%d-%H-%M", time.localtime())
if args.dataset.lower() == 'fmnist':
    args.n_channels = 1
else:
    args.n_channels = 3

if args.dataset.lower() == 'cifar100':
    args.num_classes = 100
else:
    args.num_classes = 10
criterion = nn.CrossEntropyLoss()

In [56]:
train_loader, test_loader = get_dataloader(args)

for i in range(1,4):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}.pth'))
    test_by_class(model, test_loader, i=6)

print('------------ Retrained model ------------')
for i in range(1,4):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}_r.pth'))
        test_by_class(model_r, test_loader, i=6)
    except:
        print('No retrained model')
        break

0.858, 0.984, 0.913, 0.951, 0.832, 0.974, 0.791, 0.982, 0.984, 0.965, mean_acc: 0.938
0.874, 0.985, 0.892, 0.938, 0.815, 0.970, 0.812, 0.981, 0.985, 0.968, mean_acc: 0.934
0.818, 0.984, 0.915, 0.933, 0.835, 0.963, 0.846, 0.964, 0.987, 0.978, mean_acc: 0.931
------------ Retrained model ------------
0.933, 0.976, 0.877, 0.935, 0.881, 0.971, 0.000, 0.960, 0.994, 0.966, mean_acc: 0.944
0.924, 0.984, 0.936, 0.951, 0.925, 0.956, 0.000, 0.978, 0.987, 0.967, mean_acc: 0.956
No retrained model


## Ours

In [7]:
train_targets_list = np.array(train_loader.dataset.targets)
class_loader_dict = {}
feature_list = []
merged_feat_mat = []
unlearn_class = 6
remain_class = [i for i in range(10) if i != unlearn_class]
for cls_id in range(10): 
    cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
    cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
    class_loader_dict[cls_id] = torch.utils.data.DataLoader(train_loader.dataset, 
                                                            batch_size=args.batch_size, 
                                                            sampler=cls_sampler)
    if cls_id == unlearn_class:
        continue
    for batch, (x, y) in enumerate(class_loader_dict[cls_id]):
        x = x.cuda()
        y = y.cuda()
        mat_list = get_representation_matrix(model, 
                                             x, 
                                             batch_list=[24, 100, 100, 125, 125, 250, 250, 256, 256])
        break
    threshold = 0.97 + 0.003*cls_id
    merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
    proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]


Threshold:  0.97
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/16
Layer 2 : 114/576
Layer 3 : 230/512
Layer 4 : 87/1024
Layer 5 : 42/2048
Layer 6 : 3/2048
----------------------------------------
Threshold:  0.973
Skip Updating GPM for layer: 1
Skip Updating GPM for layer: 2
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/16
Layer 2 : 114/576
Layer 3 : 259/512
Layer 4 : 149/1024
Layer 5 : 50/2048
Layer 6 : 4/2048
----------------------------------------
Threshold:  0.976
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 6/16
Layer 2 : 139/576
Layer 3 : 283/512
Layer 4 : 233/1024
Layer 5 : 85/2048
Layer 6 : 7/2048
----------------------------------------
Threshold:  0.979
Skip Updating GPM for layer: 1
----------------------------------------
Gradient Constraints Summ

Random labeling

In [14]:
for i in range(1,4):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}.pth'))
    print(f'------------ Trail {i} ------------')

    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.0005)
    unlearn_subset_loader = class_loader_dict[unlearn_class]
    
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()
            
    for ep in range(10):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            y = torch.from_numpy(np.random.choice(remain_class, size=x.shape[0])).cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        print('>> unlearned model testing acc by class SGD <<')
        test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
        test_by_class(sgd_mr_model, train_loader, i=unlearn_class)

------------ Trail 1 ------------
0.858, 0.984, 0.913, 0.951, 0.832, 0.974, 0.791, 0.982, 0.984, 0.965, mean_acc: 0.938
[train] epoch 0, batch 23, loss 5.299332141876221
>> unlearned model testing acc by class SGD <<
0.844, 0.980, 0.818, 0.925, 0.826, 0.962, 0.295, 0.925, 0.991, 0.983, mean_acc: 0.917
0.902, 0.991, 0.850, 0.960, 0.890, 0.991, 0.372, 0.945, 0.999, 0.999, mean_acc: 0.947
[train] epoch 1, batch 23, loss 3.6283650398254395
>> unlearned model testing acc by class SGD <<
0.785, 0.981, 0.698, 0.883, 0.779, 0.951, 0.124, 0.886, 0.987, 0.989, mean_acc: 0.882
0.843, 0.992, 0.728, 0.920, 0.840, 0.983, 0.159, 0.905, 0.997, 1.000, mean_acc: 0.912
[train] epoch 2, batch 23, loss 2.981870174407959
>> unlearned model testing acc by class SGD <<
0.788, 0.982, 0.696, 0.888, 0.787, 0.956, 0.075, 0.884, 0.986, 0.989, mean_acc: 0.884
0.844, 0.993, 0.739, 0.923, 0.846, 0.986, 0.090, 0.907, 0.995, 1.000, mean_acc: 0.915
[train] epoch 3, batch 23, loss 3.128439426422119
>> unlearned model tes

# Gradient Ascent

In [114]:
for i in range(1,4):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}.pth'))
    print(f'------------ Trail {i} ------------')

    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.0001)
    unlearn_subset_loader = class_loader_dict[unlearn_class]
    
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()
            
    for ep in range(10):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            y = y.cuda()
            pred_y = sgd_mr_model(x)
            loss = -criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        print('>> unlearned model testing acc by class SGD <<')
        test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
        test_by_class(sgd_mr_model, train_loader, i=unlearn_class)

------------ Trail 1 ------------
0.858, 0.984, 0.913, 0.951, 0.832, 0.974, 0.791, 0.982, 0.984, 0.965, mean_acc: 0.938
[train] epoch 0, batch 23, loss -0.5313506722450256
>> unlearned model testing acc by class SGD <<
0.875, 0.984, 0.932, 0.949, 0.856, 0.973, 0.736, 0.986, 0.984, 0.956, mean_acc: 0.944
0.953, 0.996, 0.983, 0.992, 0.937, 0.999, 0.877, 0.996, 1.000, 0.987, mean_acc: 0.982
[train] epoch 1, batch 23, loss -0.47808894515037537
>> unlearned model testing acc by class SGD <<
0.892, 0.984, 0.947, 0.946, 0.848, 0.972, 0.683, 0.987, 0.986, 0.950, mean_acc: 0.946
0.962, 0.996, 0.986, 0.988, 0.926, 0.998, 0.821, 0.996, 1.000, 0.983, mean_acc: 0.982
[train] epoch 2, batch 23, loss -0.6402273178100586
>> unlearned model testing acc by class SGD <<
0.902, 0.984, 0.950, 0.939, 0.822, 0.972, 0.611, 0.990, 0.989, 0.949, mean_acc: 0.944
0.969, 0.996, 0.987, 0.984, 0.893, 0.997, 0.731, 0.997, 1.000, 0.981, mean_acc: 0.978
[train] epoch 3, batch 23, loss -0.8684743642807007
>> unlearned m

In [116]:
for i in range(3,4):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}.pth'))
    print(f'------------ Trail {i} ------------')

    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.0001)
    unlearn_subset_loader = class_loader_dict[unlearn_class]
    
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()
            
    for ep in range(20):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            y = y.cuda()
            pred_y = sgd_mr_model(x)
            loss = -criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        print('>> unlearned model testing acc by class SGD <<')
        test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
        test_by_class(sgd_mr_model, train_loader, i=unlearn_class)

------------ Trail 3 ------------


0.818, 0.984, 0.915, 0.933, 0.835, 0.963, 0.846, 0.964, 0.987, 0.978, mean_acc: 0.931
[train] epoch 0, batch 23, loss -0.49549102783203125
>> unlearned model testing acc by class SGD <<
0.852, 0.984, 0.926, 0.933, 0.835, 0.964, 0.791, 0.965, 0.988, 0.974, mean_acc: 0.936
0.923, 0.999, 0.983, 0.985, 0.928, 0.995, 0.932, 0.985, 0.999, 0.996, mean_acc: 0.977
[train] epoch 1, batch 23, loss -0.25004032254219055
>> unlearned model testing acc by class SGD <<
0.865, 0.984, 0.932, 0.934, 0.832, 0.964, 0.770, 0.966, 0.988, 0.973, mean_acc: 0.938
0.933, 0.999, 0.985, 0.986, 0.924, 0.995, 0.912, 0.987, 0.999, 0.995, mean_acc: 0.978
[train] epoch 2, batch 23, loss -0.3200382888317108
>> unlearned model testing acc by class SGD <<
0.871, 0.984, 0.932, 0.934, 0.828, 0.962, 0.736, 0.966, 0.989, 0.972, mean_acc: 0.938
0.943, 0.999, 0.985, 0.984, 0.917, 0.995, 0.882, 0.988, 0.999, 0.994, mean_acc: 0.978
[train] epoch 3, batch 23, loss -0.45438504219055176
>> unlearned model testing acc by class SGD <<

In [25]:
for i in range(1,4):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}.pth'))
    print(f'------------ Trail {i} ------------')
        
    train_targets_list = np.array(train_loader.dataset.targets)
    class_loader_dict = {}
    feature_list = []
    merged_feat_mat = []
    unlearn_class = 6
    remain_class = [i for i in range(10) if i != unlearn_class]
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        class_loader_dict[cls_id] = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id == unlearn_class:
            continue
        for batch, (x, y) in enumerate(class_loader_dict[cls_id]):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 125, 125, 250, 250, 256, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]

    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
    test_by_class(sgd_mr_model, train_loader, i=unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.06)
    unlearn_subset_loader = class_loader_dict[unlearn_class]

    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    for ep in range(35):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            y = torch.from_numpy(np.random.choice(remain_class, size=x.shape[0])).cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            proj_mat[kk].cuda()).view(params.size())
                    kk +=1
                else:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        print('>> unlearned model testing acc by class SGD <<')
        test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
        test_by_class(sgd_mr_model, train_loader, i=unlearn_class)

------------ Trail 1 ------------
Threshold:  0.97
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 3/16
Layer 2 : 124/576
Layer 3 : 229/512
Layer 4 : 89/1024
Layer 5 : 44/2048
Layer 6 : 3/2048
----------------------------------------
Threshold:  0.973
Skip Updating GPM for layer: 2
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/16
Layer 2 : 124/576
Layer 3 : 258/512
Layer 4 : 157/1024
Layer 5 : 54/2048
Layer 6 : 4/2048
----------------------------------------
Threshold:  0.976
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/16
Layer 2 : 140/576
Layer 3 : 280/512
Layer 4 : 243/1024
Layer 5 : 90/2048
Layer 6 : 8/2048
----------------------------------------
Threshold:  0.979
Skip Updating GPM for layer: 1
----------------------------------------
Gradient Constraints S

Self-labeling

In [26]:
def get_2nd_score(model, x, y):
    indices = torch.topk(model(x), k=2, dim=1).indices
    top1_matches = indices[:, 0] == y
    selected_labels = torch.where(top1_matches, indices[:, 1], indices[:, 0])
    return selected_labels

for i in range(1,4):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}.pth'))
    print(f'------------ Trail {i} ------------')
        
    train_targets_list = np.array(train_loader.dataset.targets)
    class_loader_dict = {}
    feature_list = []
    merged_feat_mat = []
    unlearn_class = 6
    remain_class = [i for i in range(10) if i != unlearn_class]
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        class_loader_dict[cls_id] = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id == unlearn_class:
            continue
        for batch, (x, y) in enumerate(class_loader_dict[cls_id]):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 125, 125, 250, 250, 256, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]

    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
    test_by_class(sgd_mr_model, train_loader, i=unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.06)
    unlearn_subset_loader = class_loader_dict[unlearn_class]

    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(45):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            y = get_2nd_score(model, x, y.cuda())
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            proj_mat[kk].cuda()).view(params.size())
                    kk +=1
                else:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        print('>> unlearned model testing acc by class SGD <<')
        test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
        test_by_class(sgd_mr_model, train_loader, i=unlearn_class)

------------ Trail 1 ------------
Threshold:  0.97
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/16
Layer 2 : 120/576
Layer 3 : 222/512
Layer 4 : 88/1024
Layer 5 : 42/2048
Layer 6 : 3/2048
----------------------------------------
Threshold:  0.973
Skip Updating GPM for layer: 1
Skip Updating GPM for layer: 2
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/16
Layer 2 : 120/576
Layer 3 : 253/512
Layer 4 : 153/1024
Layer 5 : 51/2048
Layer 6 : 4/2048
----------------------------------------
Threshold:  0.976
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/16
Layer 2 : 138/576
Layer 3 : 275/512
Layer 4 : 238/1024
Layer 5 : 88/2048
Layer 6 : 8/2048
----------------------------------------
Threshold:  0.979
Skip Updating GPM for layer: 1
--------------------------------

In [28]:
def get_2nd_score(model, x, y):
    indices = torch.topk(model(x), k=2, dim=1).indices
    top1_matches = indices[:, 0] == y
    selected_labels = torch.where(top1_matches, indices[:, 1], indices[:, 0])
    return selected_labels

for i in range(3,4):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}.pth'))
    print(f'------------ Trail {i} ------------')
    train_targets_list = np.array(train_loader.dataset.targets)
    class_loader_dict = {}
    feature_list = []
    merged_feat_mat = []
    unlearn_class = 6
    remain_class = [i for i in range(10) if i != unlearn_class]
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        class_loader_dict[cls_id] = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id == unlearn_class:
            continue
        for batch, (x, y) in enumerate(class_loader_dict[cls_id]):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 125, 125, 250, 250, 256, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]

    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
    test_by_class(sgd_mr_model, train_loader, i=unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.06)
    unlearn_subset_loader = class_loader_dict[unlearn_class]

    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(45):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            y = get_2nd_score(model, x, y.cuda())
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            proj_mat[kk].cuda()).view(params.size())
                    kk +=1
                else:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        print('>> unlearned model testing acc by class SGD <<')
        test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
        test_by_class(sgd_mr_model, train_loader, i=unlearn_class)

------------ Trail 3 ------------
Threshold:  0.97
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/16
Layer 2 : 120/576
Layer 3 : 228/512
Layer 4 : 87/1024
Layer 5 : 37/2048
Layer 6 : 2/2048
----------------------------------------
Threshold:  0.973
Skip Updating GPM for layer: 1
Skip Updating GPM for layer: 2
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/16
Layer 2 : 120/576
Layer 3 : 259/512
Layer 4 : 154/1024
Layer 5 : 48/2048
Layer 6 : 4/2048
----------------------------------------
Threshold:  0.976
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/16
Layer 2 : 138/576
Layer 3 : 282/512
Layer 4 : 239/1024
Layer 5 : 85/2048
Layer 6 : 8/2048
----------------------------------------
Threshold:  0.979
----------------------------------------
Gradient Constraints S

In [34]:
from agents.adv import FGSM

for i in range(1,4):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}.pth'))
    print(f'------------ Trail {i} ------------')

    adv_agent = FGSM(deepcopy(model), bound=0.5, norm=False, random_start=False, device='cuda')

    def find_adjacent_cls(adv_agent, x, y):
        x_adv = adv_agent.perturb(x, y)
        adv_logits = model(x_adv)
        adv_pred = torch.argmax(adv_logits.data, 1)
        return adv_pred, x_adv

    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.0001)
    unlearn_subset_loader = class_loader_dict[unlearn_class]

    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()
            
    model.eval()
    for ep in range(35):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            adv_pred, x_adv = find_adjacent_cls(adv_agent, x, y)
            adv_y = torch.argmax(model(x_adv), dim=1).detach().cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, adv_y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        print('>> unlearned model testing acc by class SGD <<')
        test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
        test_by_class(sgd_mr_model, train_loader, i=unlearn_class)


------------ Trail 1 ------------
0.858, 0.984, 0.913, 0.951, 0.832, 0.974, 0.791, 0.982, 0.984, 0.965, mean_acc: 0.938
[train] epoch 0, batch 23, loss 5.262085437774658
>> unlearned model testing acc by class SGD <<
0.882, 0.984, 0.936, 0.944, 0.813, 0.971, 0.662, 0.977, 0.989, 0.966, mean_acc: 0.940
0.954, 0.994, 0.981, 0.988, 0.886, 0.998, 0.791, 0.992, 1.000, 0.991, mean_acc: 0.976
[train] epoch 1, batch 23, loss 2.7238504886627197
>> unlearned model testing acc by class SGD <<
0.870, 0.978, 0.883, 0.922, 0.727, 0.967, 0.503, 0.976, 0.993, 0.969, mean_acc: 0.921
0.936, 0.990, 0.919, 0.968, 0.793, 0.996, 0.624, 0.988, 1.000, 0.992, mean_acc: 0.953
[train] epoch 2, batch 23, loss 2.2510616779327393
>> unlearned model testing acc by class SGD <<
0.855, 0.973, 0.823, 0.895, 0.608, 0.964, 0.395, 0.970, 0.996, 0.972, mean_acc: 0.895
0.902, 0.983, 0.855, 0.939, 0.671, 0.993, 0.504, 0.984, 1.000, 0.993, mean_acc: 0.924
[train] epoch 3, batch 23, loss 2.2405078411102295
>> unlearned model t

In [7]:
train_targets_list = np.array(train_loader.dataset.targets)
unlearn_class = 6
cls_indices = np.where(np.isin(train_targets_list, unlearn_class))[0]
cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
unlearn_subset_loader = torch.utils.data.DataLoader(train_loader.dataset, 
                                                        batch_size=args.batch_size, 
                                                        sampler=cls_sampler)

cls_indices = np.where(~np.isin(train_targets_list, unlearn_class))[0]
cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
retain_subset_loader = torch.utils.data.DataLoader(train_loader.dataset,
                                                    batch_size=args.batch_size,
                                                    sampler=cls_sampler)

In [61]:
test_by_class(model, test_loader, i=unlearn_class)

0.818, 0.984, 0.915, 0.933, 0.835, 0.963, 0.846, 0.964, 0.987, 0.978, mean_acc: 0.931


In [17]:
def get_2nd_score(model, x, y):
    indices = torch.topk(model(x), k=2, dim=1).indices
    top1_matches = indices[:, 0] == y
    selected_labels = torch.where(top1_matches, indices[:, 1], indices[:, 0])
    return selected_labels

for i in range(1,2):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}.pth'))
    print(f'------------ Trail {i} ------------')
        
    train_targets_list = np.array(train_loader.dataset.targets)
    class_loader_dict = {}
    feature_list = []
    merged_feat_mat = []
    unlearn_class = 6
    remain_class = [i for i in range(10) if i != unlearn_class]
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        class_loader_dict[cls_id] = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id == unlearn_class:
            continue
        for batch, (x, y) in enumerate(class_loader_dict[cls_id]):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 125, 125, 250, 250, 256, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]

    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
    test_by_class(sgd_mr_model, train_loader, i=unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.06)
    unlearn_subset_loader = class_loader_dict[unlearn_class]

    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(45):
        for batch, (x, y) in enumerate(unlearn_subset_loader):
            x = x.cuda()
            y = get_2nd_score(model, x, y.cuda())
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            proj_mat[kk].cuda()).view(params.size())
                    kk +=1
                else:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        print('>> unlearned model testing acc by class SGD <<')
        test_by_class(sgd_mr_model, test_loader, i=unlearn_class)
        test_by_class(sgd_mr_model, train_loader, i=unlearn_class)

------------ Trail 1 ------------
Threshold:  0.97
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 3/16
Layer 2 : 117/576
Layer 3 : 230/512
Layer 4 : 90/1024
Layer 5 : 45/2048
Layer 6 : 3/2048
----------------------------------------
Threshold:  0.973
Skip Updating GPM for layer: 2
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/16
Layer 2 : 117/576
Layer 3 : 257/512
Layer 4 : 156/1024
Layer 5 : 55/2048
Layer 6 : 5/2048
----------------------------------------
Threshold:  0.976
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/16
Layer 2 : 136/576
Layer 3 : 275/512
Layer 4 : 241/1024
Layer 5 : 90/2048
Layer 6 : 8/2048
----------------------------------------
Threshold:  0.979
Skip Updating GPM for layer: 1
----------------------------------------
Gradient Constraints S

In [16]:
test_by_class(model_r, test_loader, i=unlearn_class)

0.933, 0.976, 0.877, 0.935, 0.881, 0.971, 0.000, 0.960, 0.994, 0.966, mean_acc: 0.944


In [3]:
train_loader, test_loader = get_dataloader(args)

for i in range(1,2):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}.pth'))
    test_by_class(model, test_loader, i=6)

print('------------ Retrained model ------------')
for i in range(1,2):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}_r.pth'))
        test_by_class(model_r, test_loader, i=6)
    except:
        print('No retrained model')
        break

0.8580, 0.9840, 0.9130, 0.9510, 0.8320, 0.9740, 0.9820, 0.9840, 0.9650, mean_acc: 0.9381
------------ Retrained model ------------
0.9330, 0.9760, 0.8770, 0.9350, 0.8810, 0.9710, 0.9600, 0.9940, 0.9660, mean_acc: 0.9437


In [11]:
import copy
import torch.nn.functional as F

def hessian(dataset, model):
    model.eval()
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)
    loss_fn = torch.nn.CrossEntropyLoss(reduction="mean")
    device = torch.device("cuda")

    for p in model.parameters():
        p.grad2_acc = 0
    
    for data, orig_target in tqdm(train_loader):
        data, orig_target = data.to(device), orig_target.to(device)
        output = model(data)
        prob = F.softmax(output, dim=-1).data

        for y in range(output.shape[1]):
            target = torch.empty_like(orig_target).fill_(y)
            loss = loss_fn(output, target)
            model.zero_grad()
            loss.backward(retain_graph=True)
            for p in model.parameters():
                if p.requires_grad:
                    p.grad2_acc += torch.mean(prob[:, y]) * p.grad.data.pow(2) 

    for p in model.parameters():
        p.grad2_acc /= len(train_loader)
    
def get_mean_var(args, p, alpha=1.25e-7):
    var = copy.deepcopy(1./(p.grad2_acc+1e-8))
    var = var.clamp(max=1e3) 
    if p.size(0) == args.num_classes:
        var = var.clamp(max=1e2)
    var = alpha * var 
    
    if p.ndim > 1:
        var = var.mean(dim=1, keepdim=True).expand_as(p).clone()
    mu = copy.deepcopy(p.data0.clone())

    if p.size(0) == args.num_classes:
        mu[unlearn_class] = 0
        var[unlearn_class] = 0.0001
        var *= 10
    elif p.ndim == 1:
        var *= 10 
    return mu, var

def fisher_new(dataset, model):
    for p in model.parameters():
        p.data0 = copy.deepcopy(p.data.clone())
    hessian(dataset, model)
    for i, p in enumerate(model.parameters()):
        mu, var = get_mean_var(args, p)
        p.data = mu + var.sqrt() * torch.empty_like(p.data).normal_()
    return model

In [5]:
unlearn_class = 6
remain_class = list(set(list(range(10))) -set([unlearn_class]))
train_targets_list = np.array(train_loader.dataset.targets)
remain_cls_indices = np.where(~np.isin(train_targets_list, unlearn_class))[0]
cls_sampler = torch.utils.data.SubsetRandomSampler(remain_cls_indices)
remain_loader = torch.utils.data.DataLoader(train_loader.dataset, 
                                            batch_size=args.batch_size, 
                                            sampler=cls_sampler)

In [22]:
fisher_model = copy.deepcopy(model)
criterion = torch.nn.CrossEntropyLoss()
remain_dataset = torch.utils.data.Subset(train_loader.dataset, remain_cls_indices)
fisher_new(remain_dataset, fisher_model)
test_by_class(fisher_model, test_loader, i=unlearn_class)

  0%|          | 7/1688 [00:00<00:24, 68.84it/s]

100%|██████████| 1688/1688 [00:20<00:00, 83.24it/s]


0.8660, 0.1360, 0.7570, 0.7100, 0.9090, 0.9630, 0.0620, 0.3850, 0.9250, 0.9830, mean_acc: 0.7371


In [23]:
train_loader, test_loader = get_dataloader(args)
remain_dataset = torch.utils.data.Subset(train_loader.dataset, remain_cls_indices)

for i in range(1,4):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}_{i}.pth'))
    test_by_class(model, test_loader, i=6)
    fisher_model = copy.deepcopy(model)
    fisher_new(remain_dataset, fisher_model)
    test_by_class(fisher_model, test_loader, i=unlearn_class)

0.8580, 0.9840, 0.9130, 0.9510, 0.8320, 0.9740, 0.7910, 0.9820, 0.9840, 0.9650, mean_acc: 0.9381


100%|██████████| 1688/1688 [00:22<00:00, 73.73it/s]


0.8380, 0.5230, 0.9720, 0.8160, 0.7210, 0.8370, 0.0020, 0.8360, 0.9460, 0.9870, mean_acc: 0.8307
0.8740, 0.9850, 0.8920, 0.9380, 0.8150, 0.9700, 0.8120, 0.9810, 0.9850, 0.9680, mean_acc: 0.9342


100%|██████████| 1688/1688 [00:22<00:00, 75.64it/s]


0.9110, 0.1970, 0.7550, 0.8350, 0.9700, 0.9510, 0.0290, 0.9380, 0.9000, 0.9560, mean_acc: 0.8237
0.8180, 0.9840, 0.9150, 0.9330, 0.8350, 0.9630, 0.8460, 0.9640, 0.9870, 0.9780, mean_acc: 0.9308


100%|██████████| 1688/1688 [00:20<00:00, 82.07it/s]


0.9900, 0.8760, 0.6560, 0.8230, 0.8000, 0.4780, 0.0000, 0.1040, 0.7990, 0.8450, mean_acc: 0.7079


In [28]:
Acc_r = 100*np.array([0.8307, 0.8237, 0.7079])
Acc_f = 100*np.array([0.0020, 0.0290, 0.0000])

print(f'Remain {np.mean(Acc_r):.4f}-{np.std(Acc_r):.4f}')
print(f'Forget {np.mean(Acc_f):.4f}-{np.std(Acc_f):.4f}')

Remain 78.7433-5.6311
Forget 1.0333-1.3225
